In [1]:
!pip install tensorflow

In [12]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [45]:
url = "https://www.basketball-reference.com/players/a/antetgi01.html"

In [46]:
response = requests.get(url)

In [47]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    headshot_elements = soup.find_all("img", {"alt": "Photo of Giannis Antetokounmpo"})
    
    # Directory to save the downloaded images
    save_directory = "headshots"
    os.makedirs(save_directory, exist_ok=True)
                
    # Download and save each headshot
    for i, headshot_element in enumerate(headshot_elements):
        img_url = urljoin(url, headshot_element["src"])
        img_data = requests.get(img_url).content
                
        with open(os.path.join(save_directory, f"headshot_{i+1}.jpg"), 'wb') as img_file:
            img_file.write(img_data)
                  
    print("Headshots downloaded")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Headshots downloaded


In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [49]:
import os

for root, dirs, files in os.walk('headshots/train'):
    for file in files:
        print(os.path.join(root, file))

headshots/train\headshot_1.jpg
headshots/train\headshot_2.jpg


In [54]:
batch_size = 2

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'headshots/train',
    target_size=(120, 180),
    batch_size=batch_size,
    class_mode='binary', 
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    'headshots/test',
    target_size=(120, 180),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False 
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [31]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid') 
])

In [32]:
model.compile(optimizer='adam',
              loss='categorial_crossentropy',
             metrics=['accuracy'])

In [33]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.

In [34]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

ValueError: Asked to retrieve element 0, but the Sequence has length 0